In [6]:
import pandas as pd

splits = {'train': 'split/train-00000-of-00001.parquet', 'validation': 'split/validation-00000-of-00001.parquet', 'test': 'split/test-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/dair-ai/emotion/" + splits["train"])

In [7]:
df.head()

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,3


In [8]:
df.label.value_counts()

label
1    5362
0    4666
3    2159
4    1937
2    1304
5     572
Name: count, dtype: int64

In [1]:
from DatasetLoad import Dataset

dataset = Dataset()
df = dataset()

In [2]:
type(df)

pandas.core.frame.DataFrame

In [3]:
df

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,3
...,...,...
15995,i just had a very brief time in the beanbag an...,0
15996,i am now turning and i feel pathetic that i am...,0
15997,i feel strong and good overall,1
15998,i feel like this was such a rude comment and i...,3


In [4]:
from TextPreprocessing import TextPreprocessing
text_preprocessing = TextPreprocessing()

In [5]:
df['processed_text'] = df['text'].apply(lambda  x: text_preprocessing.preprocess_text(x))

In [6]:
df.head()

,text,label,processed_text
0,i didnt feel humiliated,0,not feel humiliated
1,i can go from feeling so hopeless to so damned...,0,feel hopeless damned hopeful care awake
2,im grabbing a minute to post i feel greedy wrong,3,m grab minute post feel greedy wrong
3,i am ever feeling nostalgic about the fireplac...,2,feel nostalgic fireplace know property
4,i am feeling grouchy,3,feel grouchy


In [64]:
from FeatureEngineering import  FeatureEngineering

feature_engineering = FeatureEngineering(df, 100)
word2vec = feature_engineering()

In [72]:
df['vectorized_words'] = df.processed_text.apply(lambda x: feature_engineering.get_document_vectors(x))

In [73]:
df.head()

,text,label,processed_text,vectorized_words
0,i didnt feel humiliated,0,not feel humiliated,"[-0.005296618, 0.0034479387, 0.0022084336, 0.0..."
1,i can go from feeling so hopeless to so damned...,0,feel hopeless damned hopeful care awake,"[-0.0036911462, 0.002918601, 0.0017188427, 0.0..."
2,im grabbing a minute to post i feel greedy wrong,3,m grab minute post feel greedy wrong,"[-0.005523831, 0.00268102, 0.0010589175, 0.002..."
3,i am ever feeling nostalgic about the fireplac...,2,feel nostalgic fireplace know property,"[-0.0049183904, 0.0024992197, 0.002200558, 0.0..."
4,i am feeling grouchy,3,feel grouchy,"[-0.0033438364, 0.0014748467, 0.0025369273, 0...."


In [79]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.vectorized_words,
    df.label,
    stratify=df.label,
    test_size=0.2
)

In [80]:
X_train.shape

(12800,)

In [81]:
y_train.shape

(12800,)

In [82]:
from  sklearn.neighbors import KNeighborsClassifier
import numpy as np

X_train_array = np.vstack(X_train)
X_test_array = np.vstack(X_test)

model = KNeighborsClassifier(n_neighbors = 4, metric = 'euclidean')
model.fit(X_train_array, y_train)
y_pred = model.predict(X_test_array)


In [83]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.34      0.53      0.42       933
           1       0.38      0.41      0.39      1072
           2       0.13      0.07      0.09       261
           3       0.20      0.11      0.14       432
           4       0.15      0.08      0.10       387
           5       0.09      0.02      0.03       115

    accuracy                           0.32      3200
   macro avg       0.21      0.20      0.20      3200
weighted avg       0.29      0.32      0.29      3200

